<a href="https://colab.research.google.com/github/mshafi710/kttsdataenrichment/blob/main/Text_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#@title
# !pip install python-docx
# import docx
!pip install docx2txt
import docx2txt
from google.colab import drive
from collections import Counter
import pandas as pd
from matplotlib.font_manager import FontProperties
import os
import pathlib
import logging
drive.mount('/content/drive')
import nltk
nltk.download('punkt')
import re


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
from IPython.display import display, HTML
css_classes = """
<style>
  h1, h2, h3, h4 {
    color: red;
    font-size: 24px;
  }
  
  p {
    font-size: 14px;
    color: var(--text-color);
  }
</style>
"""
def  print_heading(text):
 display(HTML(f'''{css_classes}<h2> {text} </h2>'''))

def print_text(text, color='grey'):
  display (HTML(f''' {css_classes} <p style='--text-color:{color}'> {text}</p>'''))




In [62]:
WORKING_DIR= '''/content/drive/MyDrive/PhD/Dataset/Text/text_files/'''
VOCAB_FILENAME="/processed/kashmiri_vocab.csv"
ENRICHED_FILENAME_INITIAL="/processed/initial_kashmiri_enriched.csv"
ENRICHED_FILENAME_FINAL = "/processed/final_kashmiri_enriched.csv"
LOG_FILE = '/processed/text-extraction.log'
logging.basicConfig(filename=LOG_FILE, level=logging.INFO)
#first letter in the TOKEN_DELIMITER_PATTERN is the Kashmiri fullstop character  
TOKEN_DELIMITER_PATTERN = r'[۔;,:\-\s+\n+\(\)\"\?,]'
SENTENCE_DELIMITER = r'[۔]'
CORPUS_FILE = "Edited Corpus.docx"
# CORPUS_FILE = 'Corpus316.docx'
!ls $WORKING_DIR

'corpus 1.gdoc'   Corpus297.docx   Corpus308.docx   Corpus319.docx
'corpus 1.txt'	  Corpus298.docx   Corpus309.docx   Corpus320.docx
 Corpus288.docx   Corpus299.docx   Corpus310.docx   Corpus321.docx
 Corpus289.docx   Corpus300.docx   Corpus311.docx   Corpus322.docx
 Corpus290.docx   Corpus301.docx   Corpus312.docx   Corpus323.docx
 Corpus291.docx   Corpus302.docx   Corpus313.docx   Corpus324.docx
 Corpus292.docx   Corpus303.docx   Corpus314.docx   Corpus325.docx
 Corpus293.docx   Corpus304.docx   Corpus315.docx  'Edited Corpus.docx'
 Corpus294.docx   Corpus305.docx   Corpus316.docx   processed
 Corpus295.docx   Corpus306.docx   Corpus317.docx   translated_files
 Corpus296.docx   Corpus307.docx   Corpus318.docx   translated_files.tar.gz


In [63]:
def get_vocab(corpus):
  '''
  return the vocabulary list (unique words in the given corpus)  '''  

  trimmed_sentence = re.sub(TOKEN_DELIMITER_PATTERN, " ", corpus)
  tokens = trimmed_sentence.split() 
  print_heading("Overall Tokens ")
  print_text (tokens)
  print_heading("\n------------------")
  print_text("\nNo of Tokens (after excluding the punctuation): %d"  % len(tokens))
  tokens.sort()
  # get the unique list using set operation
  vocab = list(set(tokens))
  vocab.sort()
  print("Vocabulary Size: %d" %len(vocab))
  return vocab

In [64]:
def vocab_to_file(vocab):
  ''' writes vocab to the file with name in the VOCAB_FILE_NAME in the working directory  set by the variable  WORKING_DIR'''
  os.chdir(WORKING_DIR)
  with open ( WORKING_DIR + VOCAB_FILENAME,'w') as out:
      for token in vocab:
        out.write(token+"\n")
  print_text("vocabulary stored in {} \n ".format( WORKING_DIR + VOCAB_FILENAME))

In [65]:
def sentence_richness(sentence):
    '''return the basic richness of the sentence: basic richnes is 
    equal to number of unique words of a sentence'''  
    #  replaces all occurrences of this pattern with a single space
    trimmed_sentence = re.sub(TOKEN_DELIMITER_PATTERN, " ", sentence)
    tokens= trimmed_sentence.split()

    print_text("\n------In Richness ----{}----\n".format(len(tokens)),'cyan')
    print (tokens)
    print_text("\n-----Out Rchness ------------\n",'cyan')
    return  len(set(tokens))

In [ ]:
# TO DO
# If a word/senetence contains characters not in the list of kashmiri keyboard characters, reject that for simplicity purposes 
# Also delete the outliers from the dataset
corpus =  docx2txt.process(WORKING_DIR +   CORPUS_FILE)
vocab= get_vocab(corpus)
vocab_to_file(vocab)

In [67]:
def basic_richness_sentence_wise(corpus):
    sentences = re.split(SENTENCE_DELIMITER,corpus)
    print_heading('---------before processing-----------')
    print(" Total sentences: %d" % len(sentences))
    for  index, sentence  in enumerate(sentences):
      print("{}th sntence : {}".format(str(index), str(sentence)))

    '''Filter out empty sentences while doing so, also replace the multiple 
     spaces with one space, remove parenthesis, commas , and quotes and alike
      as mentioned in the token delimiter pattern'''
    sentences = [re.sub(TOKEN_DELIMITER_PATTERN, " ", sentence) for sentence in sentences if sentence.strip() ] 
    print_heading('-----------after processing---------')
    for  index, sentence  in enumerate(sentences):
      print("{}th sntence : {}".format(str(index), str(sentence)))
    print(" Total sentences: %d" % len(sentences))
    os.chdir(WORKING_DIR)  
    with open(WORKING_DIR + ENRICHED_FILENAME_INITIAL  ,'w') as out:
      out.write('sentence,richness\n')
      for  sentence in sentences:
        #exclude empty sentences
        print('Length of a sentence is {}'.format(len(sentence)))
        if len(sentence) >0:        
          basic_richness= sentence_richness(sentence)
          out.write(sentence +"," + str (basic_richness)+ "\n")
          print(sentence + ": " +str(basic_richness) + "\n-------------------\n")

In [68]:
basic_richness_sentence_wise(corpus)

In [48]:
''' reads the simple richness csv file pointed by RICHNESS_FILENAME into dataframe, sorts it  
in descending order of richness, drops the rows with NA values in richness column in place
''' 
df=pd.read_csv(WORKING_DIR+ENRICHED_FILENAME_INITIAL)
df.columns=  ['sentence','richness']
df=df.sort_values(by=['richness'], ascending=False)
#drops the old index and make the changes in the df directly
df.reset_index(drop=True, inplace=True)
#removes the dataframes where the richness column has missing or NaN values
df=df.dropna(subset=['richness'])
# add a new column to the data frame for setting the status of whether any of the tokens of the sentence has matched to the wishlist or not. 
# initially all sentences will have tokens which are in the wishlist. hence values are false.
df=df.assign(Deleted=False)
# file created for checking the temporary richness
df.to_csv(WORKING_DIR + "/processed/tmp_rich.csv")

In [ ]:
df

In [ ]:
''' calculate the richness of the sentences itertively '''
wishlist= set(vocab)
print_heading("Vocab Length:{}".format(len(wishlist)))
for index, row in df.iterrows():
    tokens= re.split(TOKEN_DELIMITER_PATTERN, row['sentence'])  
    tokens = [token for token in tokens if token.strip() ]
    tokens = set(tokens)
    print_text("Total Unique Tokens: {}".format(len(tokens)),'cyan')
    print(tokens)

    ''' richness of a given sentence is the number of unique tokens matching 
    with the remaining vocab list i.e., the intersection of the unique 
    tokens (set) in the sentence and the remaining vocab'''  

    match_words = tokens & wishlist 
    not_matched = tokens - match_words
    if len(match_words) > 0:
      df.at[index,'richness'] = len(match_words)
      print_text("\n--------match words---{}----\n".format(len(match_words)),"magenta")     
      print_text(match_words)

      print_text("\n--------Not matched words---{}----\n".format(len(not_matched)),"wheat")     
      print_text(not_matched)
      '''update the  wishlist by set difference operation of the wishlist 
      and the match  tokens: excluding the matching tokens from the wishlist'''
      wishlist = wishlist - set(match_words)
    else:
      df.at[index,'Deleted']=True

    print("Wishlist size remaining: %d " %len(wishlist) )
    print("DF size remaining: %d " %len(df) )
    print( "\n   ------------------- ")


In [ ]:
print_heading("sum of richness is {}".format(df.loc[df['Deleted'] == False, 'richness'].sum()))
print_heading ("Lenth of vocab is {}".format(len(vocab)))
df

In [15]:
df.to_csv(WORKING_DIR + ENRICHED_FILENAME_FINAL)